Crawling  
 
Architecture>
파싱해서 찾고싶은건 리스트.
링크를가지고 url list를 만들고 이걸 재귀적으로 계속 돌림
Urllist를 어덯게 관리하느냐에 따라서 성능이달라짐.

Scrapy : 크롤러
얘는 봇이야. 

Beautiful soup : 텍스트를받아서 돔 객체를 만들어주는애. 파싱하기위한 목적


    



In [1]:
#url 받아오고 에러처리

import requests
import time
def download(method, url, param=None, data=None, timeout=1, maxretries=3):
    
    try:
        resp = requests.request(method, url, params=param, data=data, headers={"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"})
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and maxretries>0:  #500대 에러일시 서버잘못이니까 재시도 메세지 띄워줌
            print(maxretries)
            time.sleep(timeout)
            resp = download(method, url, param, data, timeout, maxretries-1)
        print(e.response.status_code)
        print(e.response.reason)
    return resp
    


In [2]:
url ="http://example.webscraping.com/places/default/index"
html = download("get", url)

429
TOO MANY REQUESTS


In [3]:
from bs4 import BeautifulSoup

dom = BeautifulSoup(html.text, "lxml")
[requests.compat.urljoin(url, _["href"]) for _ in dom.find_all("a") if len(_["href"])>3]

# 샾에는 데이터가 아무것도 없음. 세자 이상만 나오도록 거르자! 
#여기까지가 구조에서 한줄 완성한거.

[]

In [84]:
#위에서 했던 동작ㅈ 똑같음. 함수로 만듯 것 
def parseURL(seed):
    urlList = list()
    
    html = download("get", seed)
    dom = BeautifulSoup(html.text, "lxml")
    #네이버븝ㄹ로그 -> 돔안에 돔이있는구조. 그래서 ㄷ안에 돔 내용은 못가져와.그안에 a태그가있는데 그걸못가져와서 에러가나는거임
    
    if len(dom.select("#mainFrame")) < 1: #바꾼부분
        return[]
    seed = requests.compat.urljoin(seed,dom.select("#mainFrame")[0]["src"]) #바꾼부분
    html = download("get", seed)
    dom = BeautifulSoup(html.text, "lxml")
    
    return [requests.compat.urljoin(url, _["href"]) for _ in dom.find_all("a") #바꾼부분
            if _.has_attr("href")==True and len(_["href"])>3 and checkBlog(requests.compat.urljoin(seed, _["href"]))]#if조건에 notstartwith #달아주기 



In [85]:
#focused crawling, 네이버 블로그에서만 돌리도록
# 만약 바꾸려면 ?! parseURL에 체크블로급부분 바꿔야하고, select부분 바꾸ㅓ야한다!
queue = list()
#queue.extend([_.find_parent()["href"] for _ in dom.select(".LC20lb")])
queue.extend([_["href"] for _ in dom.select("a.sh_blog_title._sp_each_url._sp_each_title")if checkBlog(_["href"])])
seen = list()
#queue에 어팬드 시켜서 큐에 더이상 데이터가 없을때까지 돌거임
while queue: #여긴항상트루
    baseURL = queue.pop(0)
    seen.append(baseURL)
    time.sleep(1)
    linkList = parseURL(baseURL) #여기서 중복된 유알엘들이 들어옴. 제거해야함
    for link in linkList:
        if link not in queue and link not in seen:  #링크가 queue에도 없어야하고, seen에도 없어야해
            queue.append(link)
    
    print("Queue: {0}, Seen: {1}".format(len(queue), len(seen)))  
    #baseURL.expand(linkList) #붙임

Queue: 17, Seen: 1
Queue: 29, Seen: 2
Queue: 42, Seen: 3
Queue: 41, Seen: 4
Queue: 40, Seen: 5
Queue: 39, Seen: 6
Queue: 38, Seen: 7
Queue: 37, Seen: 8
Queue: 36, Seen: 9
Queue: 35, Seen: 10
Queue: 34, Seen: 11
Queue: 33, Seen: 12
404
Not Found
Queue: 32, Seen: 13
404
Not Found
Queue: 31, Seen: 14
Queue: 54, Seen: 15
404
Not Found
Queue: 53, Seen: 16
404
Not Found
Queue: 52, Seen: 17
Queue: 51, Seen: 18
Queue: 50, Seen: 19
Queue: 49, Seen: 20
Queue: 48, Seen: 21
Queue: 47, Seen: 22


KeyboardInterrupt: 

In [63]:
url = "https://www.google.com/search"
param = {
    "q":"박보영"
}
html = download("get", url, param)
dom = BeautifulSoup(html.text, "lxml")
[_.find_parent()["href"] for _ in dom.select(".LC20lb")]

['https://ko.wikipedia.org/wiki/%EB%B0%95%EB%B3%B4%EC%98%81',
 'https://ko.wikipedia.org/wiki/%EB%B0%95%EB%B3%B4%EC%98%81%EC%9D%98_%EC%9E%91%ED%92%88_%EB%AA%A9%EB%A1%9D',
 'https://namu.wiki/w/%EB%B0%95%EB%B3%B4%EC%98%81',
 'https://news.joins.com/article/22895953',
 'https://www.msn.com/ko-kr/entertainment/news/%EB%B0%95%EB%B3%B4%EC%98%81-tvn-%EB%93%9C%EB%9D%BC%EB%A7%88-%EC%96%B4%EB%B9%84%EC%8A%A4-%EC%97%AC%EC%A3%BC%EC%9D%B8%EA%B3%B5/ar-BBLZzVh',
 'https://www.msn.com/ko-kr/entertainment/tv/hd%EC%8A%A4%ED%86%A0%EB%A6%AC-%EC%9E%91%EC%9D%80-%EA%B1%B0%EC%9D%B8-%EB%B0%95%EB%B3%B4%EC%98%81%E2%80%A6%EA%B7%B8%EC%9D%98-%EC%97%B0%EA%B8%B0%EC%97%90-%EB%8C%80%ED%95%9C-%ED%9D%94%EC%A0%81%EB%93%A4/ar-BBUvL0V',
 'https://movie.daum.net/person/main?personId=105244',
 'https://twitter.com/hashtag/%EB%B0%95%EB%B3%B4%EC%98%81']

In [59]:
queue

['https://www.google.com/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EB%8F%99%EC%9D%8C%EC%9D%B4%EC%9D%98%EC%96%B4_%EB%AC%B8%EC%84%9C',
 'https://www.google.com/wiki/%EB%B0%95%EB%B3%B4%EC%98%81_(%EB%8F%99%EC%9D%8C%EC%9D%B4%EC%9D%98)',
 'https://www.google.com/wiki/%ED%8C%8C%EC%9D%BC:%EB%B0%95%EB%B3%B4%EC%98%81,_%EB%B9%A0%EC%A0%B8%EB%93%9C%EB%8A%94_%EC%8B%AC%EC%BF%B5_%EA%BD%83%EB%AF%B8%EC%86%8C.jpg',
 'https://www.google.com/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD',
 'https://www.google.com/wiki/%EC%B6%A9%EC%B2%AD%EB%B6%81%EB%8F%84',
 'https://www.google.com/wiki/%EA%B4%B4%EC%82%B0%EA%B5%B0',
 'https://www.google.com/wiki/%EC%A6%9D%ED%8F%89%EC%9D%8D',
 'https://www.google.com/wiki/%EC%A6%9D%ED%8F%89%EA%B5%B0',
 'https://www.google.com/wiki/%EB%B0%B0%EC%9A%B0',
 'https://www.google.com/wiki/2006%EB%85%84',
 'https://www.google.com/wiki/%EB%8B%A8%EA%B5%AD%EB%8C%80%ED%95%99%EA%B5%90',
 'https://www.google.com/wiki/%EB%B0%95%EB%B3%B4%EC%98%81%EC%9D%98_%EC%88%98%EC%83%81_%EB%B0%8F_%ED%9B%84%

In [232]:
url = "https://search.daum.net/search"
param = {
    "q":"박보영"
}
html = download("get", url, param)
dom = BeautifulSoup(html.text, "lxml")
[_["href"] for _ in dom.select("#blogColl a.f_link_b") if checkBlog(_["href"])]

['http://adam24eve.tistory.com/858',
 'http://cscs0717.tistory.com/27',
 'http://listup.tistory.com/248',
 'http://34344541.tistory.com/43']

In [230]:
def checkBlog(url):
    return requests.compat.urlparse(url)[1].endswith("tistory.com")

In [231]:
checkBlog("https://adam24eve.tistory.com/858")

True

In [235]:
#다음블로그

#focused crawling, 네이버 블로그에서만 돌리도록
# 만약 바꾸려면 ?! parseURL에 체크블로급부분 바꿔야하고, select부분 바꾸ㅓ야한다!
queue = list()
#queue.extend([_.find_parent()["href"] for _ in dom.select(".LC20lb")])
queue.extend((_["href"],1)for _ in dom.select("#blogColl a.f_link_b") if checkBlog(_["href"]))
#queue.extend([_["href"] for _ in dom.select("#blogColl a.f_link_b") if checkBlog(_["href"])])
seen = list()
#queue에 어팬드 시켜서 큐에 더이상 데이터가 없을때까지 돌거임
while queue: #여긴항상트루
    baseURL = queue.pop(0)
    seen.append(baseURL)
    time.sleep(1)
    linkList = parseURL(baseURL) #여기서 중복된 유알엘들이 들어옴. 제거해야함
    for link in linkList:
        if link not in queue and link not in seen:  #링크가 queue에도 없어야하고, seen에도 없어야해
            queue.append(link)
    
    print("Queue: {0}, Seen: {1}".format(len(queue), len(seen)))  
    #baseURL.expand(linkList) #붙임

Queue: 45, Seen: 1
Queue: 59, Seen: 2
Queue: 79, Seen: 3
Queue: 84, Seen: 4
404
Not Found
Queue: 83, Seen: 5
404
Not Found
Queue: 82, Seen: 6
404
Not Found
Queue: 81, Seen: 7


KeyboardInterrupt: 

In [234]:
#parseURL함수
#가지 두개만 뻗어나가고 알아서 멈추도록
def parseURL(seed):

    html = download("get",seed[0])
    dom = BeautifulSoup(html.text,"lxml")
    
    return [(requests.compat.urljoin(url,_["href"]),seed[1]+1) 
            for _ in dom.find_all("a") 
                if seed[1] < 2 and _.has_attr("href") and  len(_["href"]) > 3
                   and checkBlog(requests.compat.urljoin(seed[0], _["href"]))]
# and not _.startwith('#')
#len(_["href"])


In [ ]:
seen

크롤링 : 네트워크 타고다니면서 링크들이 어떻게 뻗어나가는지. 나가는링크 몇개 들어오는 링크 몇개 이런거 분석하기위한 도구.
    링크정보가 가장중요해
    
스크래핑 : 그래서 내가 뽑ㅂ고싶은 데이터가 어디에 들어가있는지. 이미지면 이미지가 어디들어가있는지. 


In [153]:
#daum 주소 설정 #성진오빠꺼
url = "https://search.daum.net/search"
param = {"q":"박보영"}
html = download("get",url,param)
dom = BeautifulSoup(html.text, "lxml")

[_["href"]
    for _ in dom.select("a.f_link_b")
if checkBlog(_["href"]) ]
def parseURL(seed):
    html = download("get", seed[0])
    dom = BeautifulSoup(html.text, "lxml")
    return [requests.compat.urljoin(url,_["href"],seed[1]+1)
    for _ in dom.find_all("a") 
        if _.has_attr("href") and len(_["href"]) > 3
           seed[1] < 2 and checkBlog(requests.compat.urljoin(seed[0], _["href"]))]

#tistory.com 을 잘라내기
def checkBlog(url):
    return requests.compat.urlparse(url)[1].endswith("tistory.com")
#### 크롤링하는 코드 ########
queue = [_["href"]
    for _ in dom.select("#blogColl a.f_link_b") if checkBlog(_["href"])]
seen = list()

while queue:
    baseURL = queue.pop(0)   # 이 다음에 너무 빠르지 않게 sleep 을 추가해줘야함 
    seen.append(baseURL)
    
    time.sleep(1) #이렇게 
    linkList = parseURL(baseURL)  # linkList에 중복되는 URL 이 계속해서 생길거야
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue : {0}, Seen : {1}".format(len(queue), len(seen)))
    #queue.extend(linkList) 얘만 쓰면 중복확인이 안되서 위에

SyntaxError: invalid syntax (<ipython-input-153-5b9782d0905d>, line 16)

**DB**
#중복을 최소화하고, 테이블을 쪼갤 수 있는 만큼 쪼개야해  
table1 = id, path, param, seen, date   
table2 = id, netloc, date  
url = netloc + path + param  

In [190]:
import sqlite3
con = sqlite3.connect("bot8.db")
cur = con.cursor()

In [191]:
cur.executescript("""
    CREATE TABLE table1(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        table2_id INTEGER NOT NULL,
        path TEXT NOT NULL,
        param TEXT,
        seen BOOLEAN DEFAULT FALSE NOT NULL,
        date TIMESTAMP DEFAULT CURENT_TIMESTAMP NOT NULL
    );
    
    DROP TABLE IF EXISTS table2;
    CREATE TABLE table2(
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    netloc TEXT NOT NULL,
    date TIMESTAMP DEFAULT CURENT_TIMESTAMP NOT NULL
    );
""")
con.commit()

In [204]:
# seed URL -> DB Insert
#q를 만드는게 목적. q에 시드주소를 넣을거임.
url = "https://www.google.com/search"
param = {"q":"박보영"}
html = download("get",url,param)
dom = BeautifulSoup(html.text, "lxml")

#url하나씩 꺼내서 parse했음
for href in [_.find_parent()["href"] for _ in dom.select(".LC20lb")]:
    _urlparse = requests.compat.urlparse(href)
    netloc = "://".join(_urlparse[:2]) #netloc은 table2에넣고 나머지는 1에넣어
    cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc]) #얘가 존재하냐 안하냐 쿼리날림
    
    netlocID = cur.fetchone()#여기 값이 들어가고, NONE이면(table2에 없으면) 밑에 구문 수행함.(인서트함)
    if not netlocID:
        cur.execute("INSERT INTO table2(netloc) VALUES(?)",[netloc])
        con.commit()
        cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])
        netlocID = cur.fetchone()
        
    cur.execute("INSERT INTO table1(table2_id, path, param) VALUES(?,?,?)",
                   [netlocID[0], _urlparse[2], _urlparse[4]])
    con.commit()
    #q가 생성됨    
        


In [205]:

while True:
    cur.execute("""
        SELECT table1.id, table2.netloc,table1.path, table1.param
        FROM table1
        JOIN table2 ON table1.table2_id=table2.id
        WHERE table1.seen=FALSE 
        ORDER BY table1.date ASC 
        LIMIT 0, 1;
    """)
    
    seed = cur.fetchone()
    if not seed:
        break; #pop했음
#select해서 봤으니까 seen=TRUE로 바꿔줒ㅁ
    cur.execute("""
        UPDATE table1
        SET seen=TRUE
        WHERE id=?
    """, [seed[0]])
    con.commit()
    
    baseURL = "{0}{1}?{2}".format(seed[1],seed[2],seed[3])
    
    print(seed)
    print("{0}{1}?{2}".format(seed[1],seed[2],seed[3]))
    break

(2, 'https://ko.wikipedia.org', '/wiki/%EB%B0%95%EB%B3%B4%EC%98%81', '')
https://ko.wikipedia.org/wiki/%EB%B0%95%EB%B3%B4%EC%98%81?


In [207]:
  
while True:
    cur.execute("""
        SELECT table1.id, table2.netloc, table1.path, table1.param 
        FROM table1 JOIN table2 
        ON table1.table2_id = table2.id 
        WHERE table1.seen = FALSE
        ORDER BY table1.date ASC
        LIMIT 0,1;
    """)
    seed = cur.fetchone()
    if not seed:
        break;
    
    cur.execute("""
        UPDATE table1 SET seen=TRUE
        WHERE id=?
    """, [seed[0]])
    con.commit()
    
    baseURL = "{0}{1}?{2}".format(seed[1], seed[2], seed[3])
    for href in parseURL(baseURL):
        _urlparse = requests.compat.urlparse(href)
        netloc = "://".join(_urlparse[:2])
        cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])

        netlocID = cur.fetchone()
        if not netlocID:
            cur.execute("INSERT INTO table2(netloc) VALUES(?)",[netloc])
            con.commit()
            cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])
            netlocID = cur.fetchone()

        cur.execute("INSERT INTO table1(table2_id, path, param) VALUES(?,?,?)",
                   [netlocID[0], _urlparse[2], _urlparse[4]])
        con.commit()
    break

In [200]:
def parseURL(seed):
    html = download("get", seed)
    dom = BeautifulSoup(html.text, "lxml")
    return [requests.compat.urljoin(seed, _["href"])
 for _ in dom.find_all("a") 
        if _.has_attr("href") and len(_["href"]) > 3]